# Notes and code used on my UDACITY MLND Deep Learning module

These notes are not complete and are just a personal reference.

## Perceptron algo

This is the code. Needs data.csv and other code that runs under the hood, so I won't run it and just add it for reference.

```
import numpy as np
# Setting the random seed, feel free to change it and see different solutions.
np.random.seed(42)

def stepFunction(t):
    if t >= 0:
        return 1
    return 0

def prediction(X, W, b):
    ##return stepFunction((np.matmul(X,W)+b)[0])
    return [stepFunction(e) for e in np.matmul(X,W)+b]

# TODO: Fill in the code below to implement the perceptron trick.
# The function should receive as inputs the data X, the labels y,
# the weights W (as an array), and the bias b,
# update the weights and bias W, b, according to the perceptron algorithm,
# and return W and b.
def perceptronStep(X, y, W, b, learn_rate = 0.01):
    # Fill in code
    y_pred = prediction(X[i], W, b)
    for i in range(len(X)):
        ##y_pred = prediction(X[i], W, b)
        if y_pred[i] > y[i]:
        ##if y_pred > y[i]:
            for n in range(len(W)):
                W[n] -= learn_rate*X[i][n]
            b -= learn_rate
        elif y_pred[i] < y[i]:
        ##elif y_pred < y[i]:
            for n in range(len(W)):
                W[n] += learn_rate*X[i][n]
            b += learn_rate
    return W, b

# This function runs the perceptron algorithm repeatedly on the dataset,
# and returns a few of the boundary lines obtained in the iterations,
# for plotting purposes.
# Feel free to play with the learning rate and the num_epochs,
# and see your results plotted below.
def trainPerceptronAlgorithm(X, y, learn_rate = 0.01, num_epochs = 25):
    x_min, x_max = min(X.T[0]), max(X.T[0])
    y_min, y_max = min(X.T[1]), max(X.T[1])
    W = np.array(np.random.rand(2,1))
    b = np.random.rand(1)[0] + x_max
    # These are the solution lines that get plotted below.
    boundary_lines = []
    for i in range(num_epochs):
        # In each epoch, we apply the perceptron step.
        W, b = perceptronStep(X, y, W, b, learn_rate)
        boundary_lines.append((-W[0]/W[1], -b/W[1]))
    return boundary_lines
```

## Activation functions

Code for implementing Sigmoid function and Softmax function.

In [1]:
import numpy as np

In [3]:
'''
SIGMOID FUNCTION
'''

def sigmoid_f(x):
    if x < 0:
        z = np.exp(x)
        return z / (1+z)
    elif x >= 0:
        z = np.exp(-x)
        return 1 / (1+z)

def score(x_1, x_2):
    return 4*x_1 + 5*x_2 - 9

print(sigmoid_f(score(1,1)))
print(sigmoid_f(score(2,4)))
print(sigmoid_f(score(5,-5)))
print(sigmoid_f(score(-4,5)))

0.5
0.999999994397
8.31528027664e-07
0.5


In [3]:
'''
SOFTMAX FUNCTION
'''
def softmax(L):
    eL = []
    softmaxL = []
    for e in L:
        eL.append(np.exp(e))
    sumeL = sum(eL)
    for v in eL:
        softmaxL.append(v/float(sumeL))
    return softmaxL

L = [2, 1, 0]
print(softmax(L))

[0.6652409557748219, 0.24472847105479764, 0.090030573170380462]


In [4]:
'''Better implementation of SOFTMAX'''
def softmax_o(L):
    expL = np.exp(L)
    return np.divide (expL, expL.sum())

L = [5, 6, 7]
print(softmax_o(L))
# [0.090030573170380462, 0.24472847105479764, 0.6652409557748219]

[ 0.09003057  0.24472847  0.66524096]


The score function represent the line (plane - hyperplane) of an arbitrary neural network. The sigmoid function is a continuous differentiable function that gives back a probability of a given point to be on one or the other side of the classification boundary. A pribability of 0.5 is equivalent to having the given point right on the boundary.

The softmax function transforms a list of values to a list of probabilities that add up to one. In the case of having more than two classes, the softmax function gives the probabilities of each class.

## Cross - Entropy

If we have probabilities for each data point, we could multiply probabilities of all datapoints to get a value for a model. Since this is not a good approach because the numbers will be very small for several datapoints, we use the logarithmic property that states that the log of a multiplication is just the addition of the log of each of its constituents. Using natural log, which is negative for numbers between 0 and 1 (ln 0 approaches -inf and ln 1 = 0) we would get negative numbers, so we use a the negative of the natural log to get positive numbers and thus the expression ends up being:

-Σ[i=1, m] y_i \* ln(p_i) + (1-y_i) \* ln(1-p_i)

The smaller the cross entropy, the better the model. 

In [5]:
'''CROSS ENTROPY'''

def cross_entropy(Y, P):
    result = 0
    for i in range(len(Y)):
        result -=  Y[i]*np.log(P[i])+(1-Y[i])*np.log(1-P[i])
    return result

Y=[1,0,1,1]
P=[0.4,0.6,0.1,0.5]
print(cross_entropy(Y,P))
# 4.8283137373

4.8283137373


In [1]:
'''Provided solution'''
def cross_entropy(Y, P):
    Y = np.float_(Y)
    P = np.float_(P)
    return -np.sum(Y * np.log(P) + (1 - Y) * np.log(1 - P))


Note that the y_i is either one or zero, and there is only one vector of ys, so this is equivalent of having two classes. If we have multiple classes (n of them), the cross entropy is the following:

-Σ[i=1, m] Σ[j=1, n] y_i_j \* ln(p_i_j) 

y_i_j is 0 or 1, thus we are only adding the events that actually occured.

## Error Function

The error function for one point is:

-(1-y)\*ln(1-y_pred) - y\*ln(y_pred)

if y is equal to 1, only the second term remains; if zero only the first term remains. y_pred is the prediction, or probability, of the datapoint being of class '1' [y_pred = p(y=1)] -> according to the model.

The error function for the model then is the average of the errors:

-1/m \*  Σ[i=1, m] (1-y_i)\*ln(1-y_pred_i) + y_i\*ln(y_pred_i)

or, since y_pred = f_act(Wx+b)

-1/m \*  Σ[i=1, m] (1-y_i)\*ln(1-f_act(Wx_i+b)) + y_i\*ln(f_act(Wx_i+b))

Generalizing for more than two classes:

-1/m \*  Σ[i=1, m] Σ[j=1, n] y_i_j \* ln(y_pred_i_j)

## Gradient Descent

For a point with coordinates (x_1, ... , x_n), label y, and prediction y_pred; the gradient of the error function at that point is:

(-(y-y_pred)\*x_1, ... , -(y-y_pred)\*x_n), -(y-y_pred)) = -(y-y_pred)\*(x_1, ... , x_n)

### Gradient Descent Step

Now lets add the learning rate `alpha`

Since w_i - `alpha`(-(y-y_pred)\*x_i) = w_i + `alpha`(y-y_pred)\*x_i

w_i_new = w_i + (1/m)\*`alpha`\*x_i\*(y-y_pred)

and

b_new = b + (1/m)\*`alpha`\*(y-y_pred)

## Logistic Regression Code

Again, just for reference:

```
import numpy as np
# Setting the random seed, feel free to change it and see different solutions.
np.random.seed(42)

def sigmoid(x):
    return 1/(1+np.exp(-x))
def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))
def prediction(X, W, b):
    return sigmoid(np.matmul(X,W)+b)
def error_vector(y, y_hat):
    return [-y[i]*np.log(y_hat[i]) - (1-y[i])*np.log(1-y_hat[i]) for i in range(len(y))]
def error(y, y_hat):
    ev = error_vector(y, y_hat)
    return sum(ev)/len(ev)

# TODO: Fill in the code below to calculate the gradient of the error function.
# The result should be a list of three lists:
# The first list should contain the gradient (partial derivatives) with respect to w1
# The second list should contain the gradient (partial derivatives) with respect to w2
# The third list should contain the gradient (partial derivatives) with respect to b
def dErrors(X, y, y_hat):
    DErrorsDx1 = [(y[i]-y_hat[i])*X[i][0] for i in range(len(y))]
    DErrorsDx2 = [(y[i]-y_hat[i])*X[i][1] for i in range(len(y))]
    DErrorsDb = [(y[i]-y_hat[i]) for i in range(len(y))]
    return DErrorsDx1, DErrorsDx2, DErrorsDb

# TODO: Fill in the code below to implement the gradient descent step.
# The function should receive as inputs the data X, the labels y,
# the weights W (as an array), and the bias b.
# It should calculate the prediction, the gradients, and use them to
# update the weights and bias W, b. Then return W and b.
# The error e will be calculated and returned for you, for plotting purposes.
def gradientDescentStep(X, y, W, b, learn_rate = 0.01):
    # TODO: Calculate the prediction
    y_hat = prediction(X, W, b)
    # TODO: Calculate the gradient
    gradient = dErrors(X, y, y_hat)
    # TODO: Update the weights
    W[0] += learn_rate*sum(gradient[0])
    W[1] += learn_rate*sum(gradient[1])
    b += learn_rate*sum(gradient[2])
    # This calculates the error
    e = error(y, y_hat)
    return W, b, e

# This function runs the perceptron algorithm repeatedly on the dataset,
# and returns a few of the boundary lines obtained in the iterations,
# for plotting purposes.
# Feel free to play with the learning rate and the num_epochs,
# and see your results plotted below.
def trainLR(X, y, learn_rate = 0.01, num_epochs = 100):
    x_min, x_max = min(X.T[0]), max(X.T[0])
    y_min, y_max = min(X.T[1]), max(X.T[1])
    # Initialize the weights randomly
    W = np.array(np.random.rand(2,1))*2 -1
    b = np.random.rand(1)[0]*2 - 1
    # These are the solution lines that get plotted below.
    boundary_lines = []
    errors = []
    for i in range(num_epochs):
        # In each epoch, we apply the gradient descent step.
        W, b, error = gradientDescentStep(X, y, W, b, learn_rate)
        boundary_lines.append((-W[0]/W[1], -b/W[1]))
        errors.append(error)
    return boundary_lines, errors
```

## Neural Network Architecture

Multilayer perceptrons to achieve non linear model from the linear combination of linear models.

In [4]:
# Combination of two perceptrons:

def combination(w_1, w_2, b):
    return w_1*0.4 + w_2*0.6 + b

# Check for combination tha returns probability 0.88:
print(sigmoid_f(combination(2, 6, -2)))
print(sigmoid_f(combination(3, 5, -2.2)))
print(sigmoid_f(combination(5, 4, -3)))


0.916827303506
0.880797077978
0.802183888559


For multiclass classification you need as many outputs as there are classes.

## Feedforward and Backpropagation

### Feedforward: 

With matrix of weights from first layer is W_1, from m layer W_m. The prediction y_pred is:

y_pred = sigma o W_n o ... sigma o W_1

### Backpropagation:

Calculate derivatives of the error function using cahin rule:

The error function can be seen as a funtion on all the weights of all the layers.

E(W) = E(w1_1, w1_2, ... wm_1, ... wm_n)

Therefore the gradient is the vector fomed by all the partial derivatives:

∇E = (∂E/∂w1_1 ... ∂E/∂wm_n)

Where each component is the product of all partial derivatives. Also, having that the derivative of the sigmoid function is:

σ'(x) = σ(x)(1 - σ(x))

A given partial derivative is just the corresponding weight multiplied by the expression above.